In [ ]:
import os

In [ ]:
def save_group_runs_to_file(group: str, root_directory: str, file_name: str):
    def find_logs_in_file(file_path):
        with open(file_path, "r") as file:
            lines = file.readlines()
            result_lines = []
            for i, line in enumerate(lines):
                if "Find logs at:" in line:
                    j = i - 2
                    wandb_run_name, wandb_run_id, wandb_run_url = parse_line(
                        lines[j]
                    )
                    date, time, job_id, run_id = parse_path(file_path)
                    result_lines.append(f"wandb_run_name: {wandb_run_name}")
                    result_lines.append(f"wandb_run_id: {wandb_run_id}")
                    result_lines.append(f"wandb_run_url: {wandb_run_url}")
                    result_lines.append(f"date: {date}")
                    result_lines.append(f"time: {time}")
                    result_lines.append(f"job_id: {job_id}")
                    result_lines.append(f"run_id: {run_id}")
                    result_lines.append("-" * 88)
            return result_lines

    def parse_line(line):
        wandb_run_name = (
            line.split("wandb: 🚀 View run ")[1].split(" at:")[0].strip()
        )
        wandb_run_id = line.split("/")[-1].strip()
        wandb_run_url = line.split(" at: ")[1].strip()
        return wandb_run_name, wandb_run_id, wandb_run_url

    def parse_path(path):
        components = path.split(os.path.sep)
        date = components[-5]
        time = components[-4]
        job_run = components[-2].split("_")
        job_id = job_run[0]
        run_id = job_run[1]
        return date, time, job_id, run_id

    count = 0
    result_lines = []

    # Recursively traverse the directory structure
    for root, dirs, _ in os.walk(os.path.join(root_directory, group)):
        found_submitit = False
        if ".submitit" in dirs:
            found_submitit = True
            submitit_dir = os.path.join(root, ".submitit")
            print(f"Found submitit dir: {submitit_dir}".strip())
            result_lines.append(f"Found submitit dir: {submitit_dir}")
            result_lines.append("-" * 88)
            for sub_root, _, sub_files in os.walk(submitit_dir):
                for file in sub_files:
                    if file.endswith(".err"):
                        file_path = os.path.join(sub_root, file)
                        found_lines = find_logs_in_file(file_path)
                        if found_lines:
                            count += 1
                            result_lines.extend(found_lines)

    if found_submitit:
        with open(file_name, "w") as output_file:
            for line in result_lines:
                output_file.write(line + "\n")

        print(f"File saved to: {file_name}")

    else:
        print(f"Did not find submitit dir in: {root_directory}/{group}")
        print(f"Could not save to: {file_name}")
        result_lines.append(
            f"Did not find submitit dir in: {root_directory}/{group}"
        )
        result_lines.append("-" * 88)

    print(f"Total count: {count}")

In [ ]:
WANDB_GROUP = "gcn_wisconsin"
ROOT_DIRECTORY = "/ceph/ssd/staff/gfi/projects/cbsd/multirun"

save_group_runs_to_file(
    WANDB_GROUP,
    ROOT_DIRECTORY,
    os.path.join(ROOT_DIRECTORY, WANDB_GROUP, f"{WANDB_GROUP}.txt"),
)